# 护肤需求洞察

In [2]:
%%HTML
<button onclick="$('.input, .prompt, .output_stderr, .output_error, .output_result').toggle();">Toggle Code</button>

## 1、取数及分析方法
- 时间：10月开始，by month

- 词包：一个功效一个词包，取top1000（暂定）

- 维度：搜索词、笔记热词、笔记标题

- 品类：美妆

- 分析维度：top、sum、环比、1-10月增速、增速最值（绝对值）、变异系数

## 2、词包
### 美白：
取词：

- 功效词：美白，淡斑，祛斑，增白，变白，提亮，，……

- 成份词：烟酰胺，维C，熊果苷，谷胱甘肽，传明酸，光果甘草，甘草酰胺

- 明星产品词：小白瓶，小白灯，小灯泡……

- 医美手法：皮秒，白瓷镭射，光子嫩肤，纳晶，水光针……

- 问题词：黑色素，日晒，紫外线  

- 去除词：牙
### 抗糖：
取词：

- 功效词：抗糖，去黄（脸、面部、肤），断糖，控糖

- 成份词：肌肽、玻色因、硫辛酸、葡萄籽提取物、黄酮、多酚、阿魏酸、鞣花酸、山竹提取物、蓝莓提取物、白藜芦醇

- 明星产品词：hfp肌肽原液，奢纯

- 医美手法：

- 问题词：发黄（脸、面部、肤），变黄（脸、面部、肤），糖化，祛黄（脸、面部、肤）

- 去除词：牙，眼白，洗发水
### 抗老：
取词：

- 功效词：抗老，抗初老，防衰，抗衰，抗皱，防皱，祛皱，提拉，焕肤，紧致……

- 成份词：胜肽，视黄醇，烟酰胺……

- 明星产品词：小棕瓶……

- 医美手法：热玛吉，玻尿酸，

- 问题词：衰老，氧化，皱纹，细纹，
### 抗氧化：
取词：

- 功效词：抗氧化，防氧化，抗自由基，自由基氧化

- 成份词：葡萄籽，虾青素，花青素，茶多酚，白藜芦醇，辅酶q10，茄红素，维生素c，（氧化）维生素e，（氧化）维生素ce，维c，（氧化）维e，儿茶素，硫辛酸，谷胱甘肽，阿瑰酸

- 明星产品词：

- 医美手法：

- 问题词：（肤）氧化

### 保湿：
待补充

### 维稳：
待补充

---

## 3.1 美白搜索、笔记
### 搜索
- 取词：

功效词：美白，淡斑，祛斑，增白，变白，提亮，，……

成份词：烟酰胺，维C，熊果苷，谷胱甘肽，传明酸，光果甘草，甘草酰胺

明星产品词：小白瓶，小白灯，小灯泡……

医美手法：皮秒，白瓷镭射，光子嫩肤，纳晶，水光针……

问题词：黑色素，日晒，紫外线

去除词：牙
- 类目：美妆
- 时间：10月&9月

1. 取1-10月的数据，生成csv底表

```sql
 --搜索词搜索量降序
select
      search_keyword,
      sum(search_cnt) as search_cnt 
    from 
    (
      select
        search_keyword,
        user_token,
        taxonomy1,
        taxonomy2,
        taxonomy3,
        sum(search_cnt) as search_cnt
      from
      (
          select
            search_keyword,
            user_token,
            keyword_taxonomy1 as taxonomy1,
            keyword_taxonomy2 as taxonomy2,
            keyword_taxonomy3 as taxonomy3,
            sum(origin_query_cnt) as search_cnt
          from reddm.dm_soc_user_search_result_day_inc
          where dtm between '20201001' and '20201031'
          and dtm >= '20201101'
          and regexp_like(replace(lower(search_keyword),' ',''),'美白|淡斑|祛斑|增白|变白|提亮|烟酰胺|维C|熊果苷|谷胱甘肽|传明酸|光果甘草|甘草酰胺|小白瓶|小白灯|小灯泡|皮秒|白瓷镭射|光子嫩肤|纳晶|水光针|黑色素|日晒|紫外线')
          and not regexp_like(replace(lower(search_keyword),' ',''),'牙')
          and coalesce(keyword_taxonomy1,'') like '%美妆%'
          and coalesce(keyword_taxonomy2,'') like '%%'
          and coalesce(keyword_taxonomy3,'') like '%%'
          and type = '社区-搜索'
          group by 1,2,3,4,5
          union all
          select
            search_keyword,
            user_token,
            keyword_taxonomy1 as taxonomy1,
            keyword_taxonomy2 as taxonomy2,
            keyword_taxonomy3 as taxonomy3,
            sum(query_cnt) as search_cnt
          from reddm.dm_soc_user_search_result_day_inc
          where dtm between '20201001' and '20201031'
          and dtm < '20201101'
          and regexp_like(replace(lower(search_keyword),' ',''),'美白|淡斑|祛斑|增白|变白|提亮|烟酰胺|维C|熊果苷|谷胱甘肽|传明酸|光果甘草|甘草酰胺|小白瓶|小白灯|小灯泡|皮秒|白瓷镭射|光子嫩肤|纳晶|水光针|黑色素|日晒|紫外线')
          and not regexp_like(replace(lower(search_keyword),' ',''),'牙')
          and coalesce(keyword_taxonomy1,'') like '%美妆%'
          and coalesce(keyword_taxonomy2,'') like '%%'
          and coalesce(keyword_taxonomy3,'') like '%%'
          and type = '社区-搜索'
          group by 1,2,3,4,5
      ) a
      group by 1,2,3,4,5

    ) as a 
    -----join temp."20190802_ningli1_data_insight_user_group_1" as b
    -----on a.user_token = b.user_token
    group by 1
    order by search_cnt desc 
    limit 1000
```

2. 导入此处，将两个月的数据合并，on词

In [16]:
#数据导入
import pandas as pd
import numpy as np
df_mb_10 = pd.read_csv('美白10月.csv',header = 0 , sep = ',')
df_mb_10.rename(columns={'searchCnt':'10'}, inplace = True)
df_mb_9 = pd.read_csv('美白9月.csv',header = 0 ,sep = ',')
df_mb_9.rename(columns={'searchCnt':'9'}, inplace = True)
df_mb_8 = pd.read_csv('美白8月.csv',header = 0 ,sep = ',')
df_mb_8.rename(columns={'searchCnt':'8'}, inplace = True)
df_mb_7 = pd.read_csv('美白7月.csv',header = 0 ,sep = ',')
df_mb_7.rename(columns={'searchCnt':'7'}, inplace = True)
df_mb_6 = pd.read_csv('美白6月.csv',header = 0 ,sep = ',')
df_mb_6.rename(columns={'searchCnt':'6'}, inplace = True)
df_mb_5 = pd.read_csv('美白5月.csv',header = 0 ,sep = ',')
df_mb_5.rename(columns={'searchCnt':'5'}, inplace = True)
df_mb_4 = pd.read_csv('美白4月.csv',header = 0 ,sep = ',')
df_mb_4.rename(columns={'searchCnt':'4'}, inplace = True)
df_mb_3 = pd.read_csv('美白3月.csv',header = 0 ,sep = ',')
df_mb_3.rename(columns={'searchCnt':'3'}, inplace = True)
df_mb_2 = pd.read_csv('美白2月.csv',header = 0 ,sep = ',')
df_mb_2.rename(columns={'searchCnt':'2'}, inplace = True)
df_mb_1 = pd.read_csv('美白1月.csv',header = 0 ,sep = ',')
df_mb_1.rename(columns={'searchCnt':'1'}, inplace = True)
df_mb_10

,searchKeyword,10
0,美白,418887
1,美白精华,330528
2,光子嫩肤,215794
3,水光针,171814
4,科颜氏淡斑精华,171662
...,...,...
995,隔离美白,721
996,身体乳美白推荐,721
997,祛黄提亮面膜,721
998,光子嫩肤后注意事项,720


In [29]:
#数据合并
df_mb = pd.merge(df_mb_10,df_mb_9, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_8, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_7, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_6, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_5, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_4, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_3, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_2, how = 'outer', on = 'searchKeyword')
df_mb = pd.merge(df_mb,df_mb_1, how = 'outer', on = 'searchKeyword')

df_mb = df_mb[['searchKeyword','1','2','3','4','5','6','7','8','9','10']]
df_mb

,searchKeyword,1,2,3,4,5,6,7,8,9,10
0,美白,164864.0,482265.0,560991.0,509701.0,642854.0,613433.0,716640.0,713734.0,495300.0,418887.0
1,美白精华,173880.0,338849.0,387602.0,370623.0,367858.0,328576.0,292933.0,331049.0,263892.0,330528.0
2,光子嫩肤,87306.0,76525.0,122152.0,137188.0,138765.0,145196.0,175049.0,201885.0,203386.0,215794.0
3,水光针,136018.0,93348.0,112041.0,104614.0,99353.0,94961.0,112774.0,128975.0,157983.0,171814.0
4,科颜氏淡斑精华,55416.0,157320.0,232437.0,203366.0,169249.0,201171.0,158256.0,198019.0,145582.0,171662.0
...,...,...,...,...,...,...,...,...,...,...,...
1856,玫瑰果美白丸,366.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1857,skt小灯泡,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1858,韩国美白针,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1859,日本水光针,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# df_mb['sum'] = df_mb.apply(lambda x: x.sum(), axis = 1)
#如果要求和可以用这个，但还是不太建议，要先把第一列ky去掉。现在还不知道怎么选列，以及NaN值

In [30]:
df_mb.to_csv("美白.csv", index = False)

### 笔记
逻辑保持一致

```sql
--笔记关键词（笔记量降序）
select
  tag_name ,
  count(distinct b.discovery_id) as note_cnt
from
(
    select
        discovery_id
        ,tag_name
    from reddw.dw_soc_discovery_recommend_info_day
    where dtm = f_getdate('{{ds_nodash}}' , -1) 
    and tag_type in ('keyword','brand')
) a
join 
(
    select
        a.discovery_id
    from reddm.dm_soc_discovery_user_engagement_day_inc a
    where a.dtm =  f_getdate('{{ds_nodash}}' , -1) 
    and a.read_feed_num > 0
    and a.level > 1
    and (a.enabled is null or a.enabled=true)
    and f_getdate(a.create_time) between f_getdate('20201001') and f_getdate('20201031')
    and coalesce(a.tag_category1,'') like '%美妆%'
    and coalesce(a.tag_category2,'') like '%%'
    and coalesce(a.tag_category3,'') like '%%'
    group by 1
) b
on a.discovery_id = b.discovery_id

join
(
  select
    distinct discovery_id
  from reddw.dw_soc_discovery_content_day
  where 
    regexp_like(replace(lower(content),' ',''),'美白|淡斑|祛斑|增白|变白|提亮|烟酰胺|维C|熊果苷|谷胱甘肽|传明酸|光果甘草|甘草酰胺|小白瓶|小白灯|小灯泡|皮秒|白瓷镭射|光子嫩肤|纳晶|水光针|黑色素|日晒|紫外线')
    and not regexp_like(replace(lower(content),' ',''),'牙')
  and dtm = f_getdate('{{ds_nodash}}' , -1) 
) as c
on a.discovery_id=c.discovery_id
group by 1
order by note_cnt desc 
limit 1000
```

In [6]:
#数据导入
import pandas as pd
import numpy as np
df_mbb_10 = pd.read_csv('美白笔10月.csv',header = 0 , sep = ',')
df_mbb_10.rename(columns={'noteCnt':'10'}, inplace = True)
df_mbb_9 = pd.read_csv('美白笔9月.csv',header = 0 ,sep = ',')
df_mbb_9.rename(columns={'noteCnt':'9'}, inplace = True)
df_mbb_8 = pd.read_csv('美白笔8月.csv',header = 0 ,sep = ',')
df_mbb_8.rename(columns={'noteCnt':'8'}, inplace = True)
df_mbb_7 = pd.read_csv('美白笔7月.csv',header = 0 ,sep = ',')
df_mbb_7.rename(columns={'noteCnt':'7'}, inplace = True)
df_mbb_6 = pd.read_csv('美白笔6月.csv',header = 0 ,sep = ',')
df_mbb_6.rename(columns={'noteCnt':'6'}, inplace = True)
df_mbb_5 = pd.read_csv('美白笔5月.csv',header = 0 ,sep = ',')
df_mbb_5.rename(columns={'noteCnt':'5'}, inplace = True)
df_mbb_4 = pd.read_csv('美白笔4月.csv',header = 0 ,sep = ',')
df_mbb_4.rename(columns={'noteCnt':'4'}, inplace = True)
df_mbb_3 = pd.read_csv('美白笔3月.csv',header = 0 ,sep = ',')
df_mbb_3.rename(columns={'noteCnt':'3'}, inplace = True)
df_mbb_2 = pd.read_csv('美白笔2月.csv',header = 0 ,sep = ',')
df_mbb_2.rename(columns={'noteCnt':'2'}, inplace = True)
df_mbb_1 = pd.read_csv('美白笔1月.csv',header = 0 ,sep = ',')
df_mbb_1.rename(columns={'noteCnt':'1'}, inplace = True)
df_mbb_10

,tagName,10
0,美白,16081
1,提亮,11703
2,质地,10024
3,敏感肌,6815
4,护肤品,6584
...,...,...
995,菲诗小铺,94
996,豆沙,94
997,元气满满,93
998,整容,93


In [7]:
#数据合并
df_mbb = pd.merge(df_mbb_10,df_mbb_9, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_8, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_7, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_6, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_5, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_4, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_3, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_2, how = 'outer', on = 'tagName')
df_mbb = pd.merge(df_mbb,df_mbb_1, how = 'outer', on = 'tagName')

df_mbb = df_mbb[['tagName','1','2','3','4','5','6','7','8','9','10']]
df_mbb

,tagName,1,2,3,4,5,6,7,8,9,10
0,美白,2514.0,4801.0,8591.0,8669.0,9346.0,10284.0,11711.0,13174.0,15960.0,16081.0
1,提亮,2874.0,4346.0,7256.0,7092.0,7637.0,8734.0,9997.0,11094.0,12571.0,11703.0
2,质地,2354.0,3973.0,6836.0,6964.0,7728.0,9332.0,10413.0,10949.0,12572.0,10024.0
3,敏感肌,1229.0,2407.0,4349.0,4469.0,4969.0,5815.0,6710.0,7240.0,8138.0,6815.0
4,护肤品,1468.0,2655.0,4250.0,4071.0,4073.0,4274.0,4527.0,5162.0,6297.0,6584.0
...,...,...,...,...,...,...,...,...,...,...,...
1399,混合皮肤,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1400,makeupforever,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401,妆感自然,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1402,SK-II,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_mbb.to_csv("美白笔.csv", index = False)

-----------

## 3.0 数据处理
### 3.0.1 搜索词

In [38]:
# 优化一下语句

import pandas as pd
import numpy as np

# 数据导入
for i in range(10, 0, -1):
    locals()['df_mb_'+str(i)] = pd.read_csv('美白{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_mb_'+str(i)].rename(columns={'searchCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_mb = pd.merge(df_mb_10,df_mb_9, how = 'outer', on = 'searchKeyword')
for i in range(8, 0, -1):
    df_mb = pd.merge(df_mb, locals()['df_mb_'+str(i)], how = 'outer', on = 'searchKeyword')
df_mb = df_mb[['searchKeyword','1','2','3','4','5','6','7','8','9','10']]
df_mb

,searchKeyword,1,2,3,4,5,6,7,8,9,10
0,美白,164864.0,482265.0,560991.0,509701.0,642854.0,613433.0,716640.0,713734.0,495300.0,418887.0
1,美白精华,173880.0,338849.0,387602.0,370623.0,367858.0,328576.0,292933.0,331049.0,263892.0,330528.0
2,光子嫩肤,87306.0,76525.0,122152.0,137188.0,138765.0,145196.0,175049.0,201885.0,203386.0,215794.0
3,水光针,136018.0,93348.0,112041.0,104614.0,99353.0,94961.0,112774.0,128975.0,157983.0,171814.0
4,科颜氏淡斑精华,55416.0,157320.0,232437.0,203366.0,169249.0,201171.0,158256.0,198019.0,145582.0,171662.0
...,...,...,...,...,...,...,...,...,...,...,...
1856,玫瑰果美白丸,366.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1857,skt小灯泡,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1858,韩国美白针,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1859,日本水光针,365.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_mb.to_csv("美白.csv", index = False)

### 3.0.2 笔记词

In [40]:
import pandas as pd
import numpy as np


# 数据导入
for i in range(10, 0, -1):
    locals()['df_mbb_'+str(i)] = pd.read_csv('美白笔{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_mbb_'+str(i)].rename(columns={'noteCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_mbb = pd.merge(df_mbb_10,df_mbb_9, how = 'outer', on = 'tagName')
for i in range(8, 0, -1):
    df_mbb = pd.merge(df_mbb, locals()['df_mbb_'+str(i)], how = 'outer', on = 'tagName')
df_mbb = df_mbb[['tagName','1','2','3','4','5','6','7','8','9','10']]
df_mbb

,tagName,1,2,3,4,5,6,7,8,9,10
0,美白,2514.0,4801.0,8591.0,8669.0,9346.0,10284.0,11711.0,13174.0,15960.0,16081.0
1,提亮,2874.0,4346.0,7256.0,7092.0,7637.0,8734.0,9997.0,11094.0,12571.0,11703.0
2,质地,2354.0,3973.0,6836.0,6964.0,7728.0,9332.0,10413.0,10949.0,12572.0,10024.0
3,敏感肌,1229.0,2407.0,4349.0,4469.0,4969.0,5815.0,6710.0,7240.0,8138.0,6815.0
4,护肤品,1468.0,2655.0,4250.0,4071.0,4073.0,4274.0,4527.0,5162.0,6297.0,6584.0
...,...,...,...,...,...,...,...,...,...,...,...
1399,混合皮肤,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1400,makeupforever,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1401,妆感自然,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1402,SK-II,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_mbb.to_csv("美白笔.csv", index = False)

## 3.2抗糖
取词：

- 功效词：抗糖，去黄（脸、面部、肤），断糖，控糖

- 成份词：肌肽、玻色因、硫辛酸、葡萄籽提取物、黄酮、多酚、阿魏酸、鞣花酸、山竹提取物、蓝莓提取物、白藜芦醇

- 明星产品词：hfp肌肽原液，奢纯

- 医美手法：

- 问题词：发黄（脸、面部、肤），变黄（脸、面部、肤），糖化，AGEs，祛黄（脸、面部、肤）

- 去除词：牙，眼白，洗发水
### 3.2.1搜索词

```sql
select
      search_keyword,
      sum(search_cnt) as search_cnt 
    from 
    (

        select
          search_keyword,
          user_token,
          keyword_taxonomy1 as taxonomy1,
          keyword_taxonomy2 as taxonomy2,
          keyword_taxonomy3 as taxonomy3,
          sum(case when dtm >= '20201101' then query_cnt-full_screen_ads_query_cnt else query_cnt end) as search_cnt
        from reddm.dm_soc_user_search_result_day_inc
        where dtm between '20201001' and '20201031'
        and 
        (
        regexp_like(replace(lower(search_keyword),' ',''),'抗糖|断糖|控糖|肌肽|玻色因|硫辛酸|葡萄籽提取物|黄酮|多酚|阿魏酸|鞣花酸|山竹提取物|蓝莓提取物|白藜芦醇|hfp肌肽原液|奢纯|糖化|AGEs')
        or
        (
        regexp_like(replace(lower(search_keyword),' ',''),'去黄|祛黄|发黄|变黄')
        and
        regexp_like(replace(lower(search_keyword),' ',''),'面部|脸|肤')  
        )
        )
        and not regexp_like(replace(lower(search_keyword),' ',''),'牙|眼白|洗发水')
        and coalesce(keyword_taxonomy1,'') like '%美妆%'
        and coalesce(keyword_taxonomy2,'') like '%%'
        and coalesce(keyword_taxonomy3,'') like '%%'
        and type = '社区-搜索'
        group by 1,2,3,4,5


    ) as a 
    -----join temp."20190802_ningli1_data_insight_user_group_1" as b
    -----on a.user_token = b.user_token
    group by 1
    order by search_cnt desc 
    limit 1000
 ```

In [2]:
import pandas as pd
import numpy as np

# 数据导入
for i in range(10, 0, -1):
    locals()['df_kt_'+str(i)] = pd.read_csv('抗糖{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_kt_'+str(i)].rename(columns={'searchCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_kt = pd.merge(df_kt_10,df_kt_9, how = 'outer', on = 'searchKeyword')
for i in range(8, 0, -1):
    df_kt = pd.merge(df_kt, locals()['df_kt_'+str(i)], how = 'outer', on = 'searchKeyword')
df_kt = df_kt[['searchKeyword','1','2','3','4','5','6','7','8','9','10']]
df_kt

,searchKeyword,1,2,3,4,5,6,7,8,9,10
0,玻色因,7202.0,10562.0,14648.0,12685.0,13518.0,9811.0,8502.0,11971.0,11295.0,18275.0
1,ag抗糖面膜,15131.0,30043.0,38894.0,21370.0,17579.0,15768.0,18720.0,22511.0,20437.0,14476.0
2,抗糖面膜,5974.0,10660.0,12617.0,14432.0,7991.0,7152.0,7523.0,7569.0,8370.0,9801.0
3,优时颜抗糖精华,126.0,583.0,3671.0,2326.0,1564.0,321.0,1210.0,689.0,2461.0,7490.0
4,玻色因面霜,1642.0,2107.0,2668.0,2293.0,2236.0,1810.0,1728.0,3380.0,4033.0,6175.0
...,...,...,...,...,...,...,...,...,...,...,...
2951,日本 抗糖丸,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2952,白藜芦醇葡萄籽,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2953,EAORON澳容抗糖身体乳,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2954,兰蔻玻色因精华,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_kt.to_csv("抗糖.csv", index = False)

### 3.2.2搜索笔记

```sql
--抗糖（笔记词）
      
select
  tag_name ,
  count(distinct b.discovery_id) as note_cnt
from
(
    select
        discovery_id
        ,tag_name
    from reddw.dw_soc_discovery_recommend_info_day
    where dtm = f_getdate('{{ds_nodash}}' , -1) 
    and tag_type in ('keyword','brand')
) a
join 
(
    select
        a.discovery_id
    from reddm.dm_soc_discovery_user_engagement_day_inc a
        -----join temp."20190802_lining1_data_insight_user_group_1" as b
        -----on md5(a.user_id) = b.user_token
    where a.dtm =  f_getdate('{{ds_nodash}}' , -1) 
    and a.read_feed_num > 0
    and a.level > 1
    and (a.enabled is null or a.enabled=true)
    and f_getdate(a.create_time) between f_getdate('20201001')  and f_getdate('20201031' )
    and coalesce(a.tag_category1,'') like '%美妆%'
    and coalesce(a.tag_category2,'') like '%%'
    and coalesce(a.tag_category3,'') like '%%'
    group by 1
) b
on a.discovery_id = b.discovery_id

join
(
  select
    distinct discovery_id
  from reddw.dw_soc_discovery_content_day
  where
  (
        regexp_like(replace(lower(content),' ',''),'抗糖|断糖|控糖|肌肽|玻色因|硫辛酸|葡萄籽提取物|黄酮|多酚|阿魏酸|鞣花酸|山竹提取物|蓝莓提取物|白藜芦醇|hfp肌肽原液|奢纯|糖化|AGEs')
        or
        (
        regexp_like(replace(lower(content),' ',''),'去黄|祛黄|发黄|变黄')
        and
        regexp_like(replace(lower(content),' ',''),'面部|脸|肤')  
        )
        )
        and not regexp_like(replace(lower(content),' ',''),'牙|眼白|洗发水')
  and dtm =  f_getdate('{{ds_nodash}}' , -1) 
) as c
on a.discovery_id=c.discovery_id
group by 1
order by note_cnt desc 
limit 100000
```

In [4]:
import pandas as pd
import numpy as np


# 数据导入
for i in range(10, 0, -1):
    locals()['df_ktb_'+str(i)] = pd.read_csv('抗糖笔{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_ktb_'+str(i)].rename(columns={'noteCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_ktb = pd.merge(df_ktb_10,df_ktb_9, how = 'outer', on = 'tagName')
for i in range(8, 0, -1):
    df_ktb = pd.merge(df_ktb, locals()['df_ktb_'+str(i)], how = 'outer', on = 'tagName')
df_ktb = df_ktb[['tagName','1','2','3','4','5','6','7','8','9','10']]
df_ktb

,tagName,1,2,3,4,5,6,7,8,9,10
0,质地,460.0,888.0,1328.0,1324.0,1378.0,1621.0,1714.0,1995.0,2364.0,1809.0
1,美白,263.0,617.0,1043.0,983.0,1071.0,1136.0,1193.0,1445.0,1781.0,1453.0
2,抗氧化,300.0,576.0,931.0,867.0,940.0,1111.0,1186.0,1470.0,1709.0,1326.0
3,熬夜,230.0,408.0,624.0,656.0,798.0,894.0,925.0,1297.0,1664.0,1323.0
4,抗老,232.0,359.0,561.0,621.0,715.0,686.0,750.0,1058.0,1358.0,1160.0
...,...,...,...,...,...,...,...,...,...,...,...
10211,metabolic,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10212,brooklyn herborium,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10213,樱草面霜,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10214,迪赛,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_ktb.to_csv("抗糖笔.csv", index = False)

## 3.3抗氧化
取词：

- 功效词：抗氧化，防氧化，抗自由基，自由基氧化

- 成份词：葡萄籽，虾青素，花青素，茶多酚，白藜芦醇，辅酶q10，茄红素，维生素c，（氧化）维生素e，（氧化）维生素ce，维c，（氧化）维e，儿茶素，硫辛酸，谷胱甘肽，阿瑰酸

- 明星产品词：

- 医美手法：

- 问题词：（肤）氧化
### 3.3.1搜索词

```sql
--抗氧化（搜索词）
select
      search_keyword,
      sum(search_cnt) as search_cnt 
    from 
    (

        select
          search_keyword,
          user_token,
          keyword_taxonomy1 as taxonomy1,
          keyword_taxonomy2 as taxonomy2,
          keyword_taxonomy3 as taxonomy3,
          sum(case when dtm >= '20201101' then query_cnt-full_screen_ads_query_cnt else query_cnt end) as search_cnt
        from reddm.dm_soc_user_search_result_day_inc
        where dtm between '20201001' and '20201031'
        and
        (
        regexp_like(replace(lower(search_keyword),' ',''),'自由基|葡萄籽|虾青素|花青素|茶多酚|白藜芦醇|辅酶q10|茄红素|维生素c|维c|儿茶素|硫辛酸|谷胱甘肽|阿瑰酸')
        or
        (
        regexp_like(replace(lower(search_keyword),' ',''),'肤|抗|防|维e|维生素ce|维生素e')
        and
        regexp_like(replace(lower(search_keyword),' ',''),'氧化')
        )
        )
        and coalesce(keyword_taxonomy1,'') like '%美妆%'
        and coalesce(keyword_taxonomy2,'') like '%%'
        and coalesce(keyword_taxonomy3,'') like '%%'
        and type = '社区-搜索'
        group by 1,2,3,4,5


    ) as a 
    -----join temp."20190802_ningli1_data_insight_user_group_1" as b
    -----on a.user_token = b.user_token
    group by 1
    order by search_cnt desc 
    limit 1000
```

In [6]:
import pandas as pd
import numpy as np

# 数据导入
for i in range(10, 0, -1):
    locals()['df_ky_'+str(i)] = pd.read_csv('抗氧{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_ky_'+str(i)].rename(columns={'searchCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_ky = pd.merge(df_ky_10,df_ky_9, how = 'outer', on = 'searchKeyword')
for i in range(8, 0, -1):
    df_ky = pd.merge(df_ky, locals()['df_ky_'+str(i)], how = 'outer', on = 'searchKeyword')
df_ky = df_ky[['searchKeyword','1','2','3','4','5','6','7','8','9','10']]
df_ky

,searchKeyword,1,2,3,4,5,6,7,8,9,10
0,虾青素,40823.0,92331.0,102349.0,79373.0,67786.0,53098.0,52007.0,56412.0,47859.0,51568.0
1,抗氧化精华,23209.0,57580.0,53769.0,44120.0,44202.0,40309.0,37371.0,42251.0,41105.0,49500.0
2,德妃虾青素水乳,NaN,392.0,5330.0,27645.0,35302.0,27650.0,24574.0,40069.0,40953.0,40494.0
3,抗氧化,8222.0,36118.0,43927.0,35272.0,40834.0,36668.0,33894.0,33479.0,33936.0,33327.0
4,haa虾青素,9049.0,9411.0,26930.0,23381.0,23278.0,11585.0,10540.0,5230.0,11728.0,21190.0
...,...,...,...,...,...,...,...,...,...,...,...
1999,辅酶q10怎么吃,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,韩国济州岛维c,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,亲测好用的科颜氏高浓度维C精华,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,葡萄籽和蔓越莓,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_ky.to_csv("抗氧.csv", index = False)

## 3.3.2笔记词
```sql
-- 抗氧化（笔记词）
select
  tag_name ,
  count(distinct b.discovery_id) as note_cnt
from
(
    select
        discovery_id
        ,tag_name
    from reddw.dw_soc_discovery_recommend_info_day
    where dtm = f_getdate('{{ds_nodash}}' , -1) 
    and tag_type in ('keyword','brand')
) a
join 
(
    select
        a.discovery_id
    from reddm.dm_soc_discovery_user_engagement_day_inc a
        -----join temp."20190802_lining1_data_insight_user_group_1" as b
        -----on md5(a.user_id) = b.user_token
    where a.dtm =  f_getdate('{{ds_nodash}}' , -1) 
    and a.read_feed_num > 0
    and a.level > 1
    and (a.enabled is null or a.enabled=true)
    and f_getdate(a.create_time) between f_getdate('20201001')  and f_getdate('20201031' )
    and coalesce(a.tag_category1,'') like '%美妆%'
    and coalesce(a.tag_category2,'') like '%%'
    and coalesce(a.tag_category3,'') like '%%'
    group by 1
) b
on a.discovery_id = b.discovery_id

join
(
  select
    distinct discovery_id
  from reddw.dw_soc_discovery_content_day
  where
        (
        regexp_like(replace(lower(content),' ',''),'自由基|葡萄籽|虾青素|花青素|茶多酚|白藜芦醇|辅酶q10|茄红素|维生素c|维c|儿茶素|硫辛酸|谷胱甘肽|阿瑰酸')
        or
        (
        regexp_like(replace(lower(content),' ',''),'肤|抗|防|维e|维生素ce|维生素e')
        and
        regexp_like(replace(lower(content),' ',''),'氧化')
        )
        )
  and dtm =  f_getdate('{{ds_nodash}}' , -1) 
) as c
on a.discovery_id=c.discovery_id
group by 1
order by note_cnt desc 
limit 1000
```

In [10]:
import pandas as pd
import numpy as np


# 数据导入
for i in range(10, 0, -1):
    locals()['df_kyb_'+str(i)] = pd.read_csv('抗氧笔{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_kyb_'+str(i)].rename(columns={'noteCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_kyb = pd.merge(df_kyb_10,df_kyb_9, how = 'outer', on = 'tagName')
for i in range(8, 0, -1):
    df_kyb = pd.merge(df_kyb, locals()['df_kyb_'+str(i)], how = 'outer', on = 'tagName')
df_kyb = df_kyb[['tagName','1','2','3','4','5','6','7','8','9','10']]
df_kyb

,tagName,1,2,3,4,5,6,7,8,9,10
0,抗氧化,1411.0,2667.0,4257.0,4160.0,4395.0,4883.0,5215.0,6506.0,8338.0,6490.0
1,质地,986.0,1713.0,2616.0,2668.0,2920.0,3304.0,3616.0,4201.0,5181.0,3518.0
2,氧化,724.0,1130.0,1849.0,1837.0,1937.0,2012.0,2169.0,2582.0,3064.0,2476.0
3,美白,536.0,1089.0,1872.0,1829.0,1941.0,2158.0,2349.0,2523.0,3081.0,2345.0
4,敏感肌,512.0,995.0,1691.0,1800.0,1835.0,2074.0,2277.0,2533.0,3133.0,2215.0
...,...,...,...,...,...,...,...,...,...,...,...
1487,concealer,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1488,护肤保养,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1489,露华浓,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1490,黑茶,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_kyb.to_csv("抗氧笔.csv", index = False)

# 3.4抗老化
取词：

- 功效词：抗老，抗初老，防衰，抗衰，抗皱，防皱，祛皱，提拉，焕肤，紧致……

- 成份词：胜肽，视黄醇，烟酰胺……

- 明星产品词：小棕瓶……

- 医美手法：热玛吉，玻尿酸，

- 问题词：衰老，氧化，皱纹，细纹，
## 3.4.1搜素词

```sql
select
      search_keyword,
      sum(search_cnt) as search_cnt 
    from 
    (

        select
          search_keyword,
          user_token,
          keyword_taxonomy1 as taxonomy1,
          keyword_taxonomy2 as taxonomy2,
          keyword_taxonomy3 as taxonomy3,
          sum(case when dtm >= '20201101' then query_cnt-full_screen_ads_query_cnt else query_cnt end) as search_cnt
        from reddm.dm_soc_user_search_result_day_inc
        where dtm between '20200101' and '20200131'
        and
        (
        regexp_like(replace(lower(search_keyword),' ',''),'抗老|抗初老|防衰|抗衰|抗皱|防皱|祛皱|提拉|焕肤|紧致|胜肽|视黄醇|烟酰胺|热玛吉|肉毒素|衰老|皱纹|细纹')
        or
        (
        regexp_like(replace(lower(search_keyword),' ',''),'小棕瓶')
        and not regexp_like(replace(lower(search_keyword),' ',''),'英诺')
        )
        )
        and coalesce(keyword_taxonomy1,'') like '%美妆%'
        and coalesce(keyword_taxonomy2,'') like '%%'
        and coalesce(keyword_taxonomy3,'') like '%%'
        and type = '社区-搜索'
        group by 1,2,3,4,5


    ) as a 
    -----join temp."20190802_ningli1_data_insight_user_group_1" as b
    -----on a.user_token = b.user_token
    group by 1
    order by search_cnt desc 
    limit 1000
```

In [8]:
import pandas as pd
import numpy as np

# 数据导入
for i in range(10, 0, -1):
    locals()['df_kl_'+str(i)] = pd.read_csv('抗老{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_kl_'+str(i)].rename(columns={'searchCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_kl = pd.merge(df_kl_10,df_kl_9, how = 'outer', on = 'searchKeyword')
for i in range(8, 0, -1):
    df_kl = pd.merge(df_kl, locals()['df_kl_'+str(i)], how = 'outer', on = 'searchKeyword')
df_kl = df_kl[['searchKeyword','1','2','3','4','5','6','7','8','9','10']]
df_kl

,searchKeyword,1,2,3,4,5,6,7,8,9,10
0,雅诗兰黛小棕瓶,214940.0,289667.0,320472.0,295814.0,338191.0,431457.0,326564.0,291233.0,209107.0,314888.0
1,小棕瓶,86079.0,110819.0,124482.0,126586.0,155172.0,239071.0,184409.0,141830.0,91609.0,265638.0
2,热玛吉,84232.0,51994.0,98112.0,143517.0,184308.0,240745.0,303132.0,375490.0,298832.0,249319.0
3,第七代小棕瓶,NaN,NaN,NaN,NaN,NaN,NaN,19149.0,54050.0,11238.0,163301.0
4,抗老精华,60511.0,87967.0,95264.0,93047.0,84663.0,88607.0,83090.0,98674.0,83348.0,139724.0
...,...,...,...,...,...,...,...,...,...,...,...
1922,pola祛皱,222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1923,瘦脸提拉针,221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1924,百瑟烟酰胺,221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1925,丽芝雅烟酰胺,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_kl.to_csv("抗老.csv", index = False)

## 3.4.2笔记词
```sql
-- 抗老（笔记词）

select
  tag_name ,
  count(distinct b.discovery_id) as note_cnt
from
(
    select
        discovery_id
        ,tag_name
    from reddw.dw_soc_discovery_recommend_info_day
    where dtm = f_getdate('{{ds_nodash}}' , -1) 
    and tag_type in ('keyword','brand')
) a
join 
(
    select
        a.discovery_id
    from reddm.dm_soc_discovery_user_engagement_day_inc a
        -----join temp."20190802_lining1_data_insight_user_group_1" as b
        -----on md5(a.user_id) = b.user_token
    where a.dtm =  f_getdate('{{ds_nodash}}' , -1) 
    and a.read_feed_num > 0
    and a.level > 1
    and (a.enabled is null or a.enabled=true)
    and f_getdate(a.create_time) between f_getdate('20201001')  and f_getdate('20201031' )
    and coalesce(a.tag_category1,'') like '%美妆%'
    and coalesce(a.tag_category2,'') like '%%'
    and coalesce(a.tag_category3,'') like '%%'
    group by 1
) b
on a.discovery_id = b.discovery_id

join
(
  select
    distinct discovery_id
  from reddw.dw_soc_discovery_content_day
  where
       (
        regexp_like(replace(lower(content),' ',''),'抗老|抗初老|防衰|抗衰|抗皱|防皱|祛皱|提拉|焕肤|紧致|胜肽|视黄醇|烟酰胺|热玛吉|肉毒素|衰老|皱纹|细纹')
        or
        (
        regexp_like(replace(lower(content),' ',''),'小棕瓶')
        and not regexp_like(replace(lower(content),' ',''),'英诺')
        )
        )
  and dtm =  f_getdate('{{ds_nodash}}' , -1) 
) as c
on a.discovery_id=c.discovery_id
group by 1
order by note_cnt desc 
limit 1000
```

In [12]:
import pandas as pd
import numpy as np


# 数据导入
for i in range(10, 0, -1):
    locals()['df_klb_'+str(i)] = pd.read_csv('抗老笔{}月.csv'.format(i),header = 0 , sep = ',')
    locals()['df_klb_'+str(i)].rename(columns={'noteCnt':'{}'.format(i)}, inplace = True)

# 数据合并
df_klb = pd.merge(df_klb_10,df_klb_9, how = 'outer', on = 'tagName')
for i in range(8, 0, -1):
    df_klb = pd.merge(df_klb, locals()['df_klb_'+str(i)], how = 'outer', on = 'tagName')
df_klb = df_klb[['tagName','1','2','3','4','5','6','7','8','9','10']]
df_klb

,tagName,1,2,3,4,5,6,7,8,9,10
0,紧致,1965.0,3022.0,4791.0,4938.0,5386.0,6095.0,7555.0,8936.0,10952.0,10384.0
1,质地,2612.0,4175.0,6367.0,6544.0,7258.0,8370.0,9231.0,10645.0,13228.0,10216.0
2,抗老,1452.0,2062.0,3244.0,3483.0,3901.0,4342.0,4968.0,6309.0,8386.0,7715.0
3,护肤品,2041.0,3232.0,4892.0,4477.0,4608.0,4709.0,4900.0,5897.0,7575.0,7400.0
4,滋润,1958.0,2977.0,4265.0,4068.0,4350.0,4968.0,5637.0,5964.0,7270.0,7101.0
...,...,...,...,...,...,...,...,...,...,...,...
1384,贝玲妃,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1385,毁容,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1386,canmake,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387,本土品牌,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_klb.to_csv("抗老笔.csv", index = False)